In [ ]:
library(keras)  # FashionMNIST dataset
library(nnet)  # Neural networks
library(caret)  # Cross Validation - loads nnet directly on trainControl

library(doParallel) # Parallel cross-validation. 

# Dataset

First we load the dataset from keras package. _Check legacyLoad.R to see how to load the dataset without using the package._

In [ ]:
fashion <- dataset_fashion_mnist()

In [ ]:
str(fashion)
attach(fashion) # So we can access test and train directly!

We get the following structure:

- train: Training dataset
    + x: the predictors, 28x28 pixels image in grayscale.
    + y: the response
- test: Testing datset (with x and y)

We can see the images with the following function:

## dataset visualization

In [ ]:
rotate <- function(x) t(apply(x, 2, rev))
show_image <- function(imgarray, col=gray(12:1/12), ...) {
  image(rotate(matrix(imgarray, nrow=28)), col=col, ...)
}

show_image(train$x[2,,])
show_image(train$x[10,,])

## Response reencode

Notice that in y we have an integer from 0 to 9 (10 classes). They are in fact the following:
- 0: T-shirt/top
- 1: Trouser
- 2: Pullover
- 3: Dress
- 4: Coat
- 5: Sandal
- 6: Shirt
- 7: Sneaker
- 8: Bag
- 9: Ankle boot

We recode the response variable to factor.

In [ ]:
classString <- c("T-shirt/top","Trouser", "Pullover", "Dress", "Coat", "Sandal",
              "Shirt","Sneaker", "Bag","Ankle boot")

# y+1 because 0 is the first class and in R we start indexing at 1!
train$yFactor <- as.factor(classString[train$y+1]) 
test$yFactor <- as.factor(classString[test$y+1])

For the CNN we use one hot encoding to produce a vector of 10 values per sample, with a one on the class (probablity of belonging to a given class).

In [ ]:
train$yOneHot <- class.ind(train$yFactor)
test$yOneHot <- class.ind(test$yFactor)

In [ ]:
str(train$yOneHot)
train$y[1:10]
train$yOneHot[1:10,]

*class.ind* reorders the classes alfabetically, therefore we need to revert this order to the original provided. We use *match* over the column names to get a vector of the reorder to match the column names to **classString**.

In [ ]:
colnames(train$yOneHot)
classString
(m <- match(classString, colnames(test$yOneHot))) 

In [ ]:
train$yOneHot <- train$yOneHot[,m]
test$yOneHot <- test$yOneHot[,m]

Now the order is correct

In [ ]:
colnames(train$yOneHot)
classString

## Add missing dimension

Convolutional layers will expect the input to have 4 dimensions:
- Sample dimension
- Height dimension
- Width dimension
- Channel dimension

In our case we have only one channel as the image is grayscale. If it's a color image we would have 3 or 4 channels (Red, Green, Blue and Alpha (transparency)). We need to add the missing dimension, however this will not modify the data. 

In [ ]:
dim(train$x) <- c(dim(train$x),1)
dim(test$x) <- c(dim(test$x),1)

## Create a dataset for nnet

Now we prepare join the X and the Y in a data.frame.

In [ ]:
nnetData <- data.frame(train$x, class=train$yFactor)
nnetDataTest <- data.frame(test$x, class=test$yFactor)

# Training a Neural Network

We can train the model directly as follows, but we will use _caret's_
_trainControl_ for CrossValidation.

In [ ]:
model.nnet <- nnet(class ~ ., data=nnetData, size=50, maxit=300,decay=0.5, MaxNWts = 39760)

Specifically a 5 fold cross-validation. We don't go for a 10 fold
cross-validation as it will take a lot of time to compute.

In [ ]:
## specify 5-CV
K <- 5
trc <- trainControl (method="repeatedcv", number=K, repeats=1)
(decays <- 10^seq(-3,0,by=0.25))

We now specify that we want to execute the cross validation in parallel:

In [ ]:
# Use all cores except one (recommended if you want to use your computer for something else). Or half of the lenght of the decays (RAM issues).
cores <- min(detectCores()-1, ceiling(length(decays)/2))
registerDoParallel(cores = cores) 

Beware with the number of cores used, it will impact in the RAM usage. ~10GB per thread with 60K Fashion MNIST samples. **Don't execute this training if you don't have a big machine, just load the model.**

The cross-validation process will take about 30 hours using 7 cores of a Intel(R) 
Xeon(R) CPU E5-2630 v4 @ 2.20GHz and about 80 GB of RAM.

Remember that we're training (number_of_decay_param x number_of_folds) = 14 x 5
= 70 models.

In [ ]:
## WARNING: this takes some time
model.5CV <- train (class ~ ., data=nnetData, method='nnet', maxit = 300, trace = FALSE,
                      tuneGrid = expand.grid(.size=50,.decay=decays), trControl=trc, MaxNWts=39760)

In [ ]:
# Save model
save(model.5CV, file="nnet.mod")

In [ ]:
load("nnet.mod")

In [ ]:
model.5CV

The best model we got has an accuracy of 84%. Not bad at all for a 10 class
classification problem.

In [ ]:
pred <- predict(model.5CV,nnetDataTest)
(t <- table(nnetDataTest$class, pred))

In [ ]:
(accuracy <- sum(diag(t))/sum(t))

Test also gives us 84% of accuracy. Nice model :)

# Convolutional Neural Networks

## Model architecture definition: LeNet

Now we have to define the CNN architecture. In this case we use LeNet, proposed
by LeCun et al. (Gradient-based learning applied to document recognition. 
Proceedings of the IEEE, november 1998). 

It is composed by two packs of convolutional-activation(tanh)-pooling layers and
two fully connected layers with a softmax layer at the end.

In Keras, as in most of the packages, we define layers as objects and the 
connections between those objects. In this case we implicitly connect everything
using the %>% operator.

In [ ]:
lenet <- keras_model_sequential() %>%
    # First convolutional block
    layer_conv_2d(filters=20, kernel_size=c(5,5), activation="tanh",
        input_shape=c(28,28,1), padding="same") %>%  # We define here the input size
    layer_max_pooling_2d(pool_size=c(2,2),strides=c(2,2)) %>%
    # Second convolutional block
    layer_conv_2d(filters=50, kernel_size=c(5,5), activation="tanh",
        input_shape=c(28,28,1), padding="same") %>% 
    layer_max_pooling_2d(pool_size=c(2,2),strides=c(2,2)) %>%
    
    # Flatten the matrix to a vector for the fully connected layers
    layer_flatten() %>%

    # First fully connected block
    layer_dense(units=500, activation="tanh") %>%
    # Second fully connected block
    layer_dense(units=10, activation="softmax") 
    # This last layer will produce the final classification (probability of 
    # belonging to a class). 10 different units, 10 different classes.

Now we check the architecture we have defined:

In [ ]:
lenet

Notice that we're adjusting 1 million parameters this time. With the nnet one layer network we were training just 39.760 parameters.

Last thing we have to do is to specify which optimizaton algorithm and metrics
we want to use with the compile step.

In [ ]:
sgd <- optimizer_sgd(
                     lr=0.05, 
                     decay=0.001, 
                     momentum=0.8, 
                     clipnorm=1.
)
lenet %>% compile(optimizer=sgd,
              loss='categorical_crossentropy', 
              metrics = "accuracy"
)

## Model training

Now we're going to train the network using CPU (if you're not using
tensorflow-gpu). Mind that if you want to use GPUs you need to have the GPU 
version of the package and the required Nvidia packages (check PlaidML for 
non-Nvidia GPUs). 

In [ ]:
lenet %>% fit(
          train$x, 
          train$yOneHot, 
          batch_size=50,
          validation_split=0.2,
          epochs=10
)

Takes about 3 minuts with 40 cores and 18 GB of RAM. It may take less with GPUs!

And now we save the trained model for convenience:

In [ ]:
lenet %>% save_model_hdf5("lenet-FashionMNIST.h5")

## Predicting using the model

Predicting the label for the test set

In [ ]:
lenet <- load_model_hdf5("lenet-FashionMNIST.h5")

In [ ]:
lenet

In [ ]:
pred_prob <- predict(lenet, test$x)

In [ ]:
head(pred_prob)

For each element we get the probability of that element to be of each class, therefore we search for the value that is maximum in each row and then we create the confusion matrix.

In [ ]:
predClass <- apply(pred_prob,1,which.max)
predClass <- classString[predClass] # And change the integers by their class tag


trueClass <- test$yFactor

# Now we do a confusion matrix and analyze it
(cMatrix <- table(trueClass,predClass))

In [ ]:
correctClass <- sum(diag(cMatrix))
total <- sum(cMatrix)
(accuracy <- correctClass/total)

We're getting about a 90% of accuracy that may be improved with further tunning of the network. Notice that, for example, there are 30 ankle boots classified as sneakers, that can have similar shapes. Also there are 113 pullovers are classified as coats.